<a href="https://colab.research.google.com/github/dbenayoun/IASD/blob/main/Copy_of_importGolois.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://www.lamsade.dauphine.fr/~cazenave/project2022.zip
!unzip project2022.zip

--2024-06-03 18:35:02--  https://www.lamsade.dauphine.fr/~cazenave/project2022.zip
Resolving www.lamsade.dauphine.fr (www.lamsade.dauphine.fr)... 193.48.71.250
Connecting to www.lamsade.dauphine.fr (www.lamsade.dauphine.fr)|193.48.71.250|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 138884468 (132M) [application/zip]
Saving to: ‘project2022.zip’

project2022.zip     100%[===================>] 132.45M  21.3MB/s    in 7.2s    

2024-06-03 18:35:10 (18.4 MB/s) - ‘project2022.zip’ saved [138884468/138884468]

Archive:  project2022.zip
  inflating: Board.h                 
  inflating: Game.h                  
  inflating: Rzone.h                 
  inflating: compile.sh              
  inflating: compileMAC.sh           
  inflating: games.data              
  inflating: golois.cpp              
  inflating: golois.cpython-310-x86_64-linux-gnu.so  
  inflating: golois.cpython-37m-x86_64-linux-gnu.so  
  inflating: golois.cpython-38-x86_64-linux-gnu.so  
  inflat

In [ ]:
!python golois.py

2024-05-31 09:50:28.559491: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-31 09:50:28.559546: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-31 09:50:28.561592: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-31 09:50:28.577077: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-31 09:50:30.383233: W tensorflow/compiler/tf2

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import gc
import pandas as pd
from keras.callbacks import LearningRateScheduler


In [ ]:
import golois


In [ ]:
planes = 31
moves = 361
N = 10000
epochs = 50
batch = 128
filters = 32

In [ ]:
input_data = np.random.randint(2, size=(N, 19, 19, planes))
input_data = input_data.astype ('float32')

policy = np.random.randint(moves, size=(N,))
policy = keras.utils.to_categorical (policy)

value = np.random.randint(2, size=(N,))
value = value.astype ('float32')

end = np.random.randint(2, size=(N, 19, 19, 2))
end = end.astype ('float32')

groups = np.zeros((N, 19, 19, 1))
groups = groups.astype ('float32')

print ("getValidation", flush = True)
golois.getValidation (input_data, policy, value, end)

getValidation


# Best Model so far

In [ ]:
input = keras.Input(shape=(19, 19, planes), name='board')
x = layers.Conv2D(filters, 1, activation='relu', padding='same')(input)
x = layers.BatchNormalization()(x)

for i in range (4):
    ident = x
    x = layers.Conv2D(filters, 3, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x) #added batch normalization
    x = layers.Conv2D(filters, 3, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x) #added batch normalization
    x = layers.add([x, ident])

# Policy head
policy_head = layers.Conv2D(1, 1, activation='relu', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(x)
policy_head = layers.BatchNormalization()(policy_head)
policy_head = layers.Flatten()(policy_head)
policy_head = layers.Activation('softmax', name='policy')(policy_head)

# Value head
value_head = layers.Conv2D(1, 1, activation='relu', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(x)
value_head = layers.BatchNormalization()(value_head)
value_head = layers.Flatten()(value_head)
value_head = layers.Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(value_head)
value_head = layers.Dropout(0.5)(value_head)
value_head = layers.Dense(1, activation='sigmoid', name='value', kernel_regularizer=regularizers.l2(0.0001))(value_head)

model = keras.Model(inputs=input, outputs=[policy_head, value_head])



In [ ]:
model.count_params()

94383

# Testing Model


In [ ]:
input = keras.Input(shape=(19, 19, planes), name='board')
x = layers.Conv2D(filters, 1, activation='relu', padding='same')(input)
x = layers.BatchNormalization()(x)

def bottleneck_block(x, expand=96, squeeze=32):
  m = layers.Conv2D(expand, (1,1), kernel_regularizer=regularizers.l2(0.0001), use_bias = False)(x)
  m = layers.BatchNormalization()(m)
  m = layers.Activation('relu')(m)
  m = layers.DepthwiseConv2D((3,3), padding='same', kernel_regularizer=regularizers.l2(0.0001), use_bias = False)(m)
  m = layers.BatchNormalization()(m)
  m = layers.Activation('relu')(m)
  m = layers.Conv2D(squeeze, (1,1), kernel_regularizer=regularizers.l2(0.0001), use_bias = False)(m)
  m = layers.BatchNormalization()(m)
  return layers.Add()([m, x])

def convnext(x, filters):
  x1 = layers.DepthwiseConv2D((7,7), padding='same', use_bias = False)(x)
  x1 = layers.LayerNormalization()(x1)
  x1 = layers.Conv2D(4 * filters, 1, padding='same', activation='gelu')(x1)
  x1 = layers.Conv2D(filters, 1, padding='same')(x1)
  x = layers.add([x1,x])
  return x


for i in range (8):
  x = convnext(x, filters)
x = tf.keras.layers.BatchNormalization()(x)

# Policy head
policy_head = layers.Conv2D(1, 1, activation='relu', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(x)
policy_head = layers.BatchNormalization()(policy_head)
policy_head = layers.Flatten()(policy_head)
policy_head = layers.Activation('softmax', name='policy')(policy_head)

# Value head
value_head = layers.Conv2D(1, 1, activation='relu', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(x)
value_head = layers.BatchNormalization()(value_head)
value_head = layers.Flatten()(value_head)
value_head = layers.Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(value_head)
value_head = layers.Dropout(0.5)(value_head)
value_head = layers.Dense(1, activation='sigmoid', name='value', kernel_regularizer=regularizers.l2(0.0001))(value_head)

model = keras.Model(inputs=input, outputs=[policy_head, value_head])



In [ ]:
model.count_params()

99375

1. added a residual layer


# Summary

In [ ]:
model.summary ()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 board (InputLayer)          [(None, 19, 19, 31)]         0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 19, 19, 32)           1024      ['board[0][0]']               
                                                                                                  
 batch_normalization (Batch  (None, 19, 19, 32)           128       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 conv2d_1 (Conv2D)           (None, 19, 19, 32)           9248      ['batch_normalization[0][0

# Compile

In [ ]:
planes = 31
moves = 361
N = 10000
epochs = 50
batch = 128
filters = 32
lr = 0.0005

In [ ]:
def step_decay(epoch):
  #use cosine
    initial_lr = 0.0005
    drop = 0.5
    epochs_drop = 10.0
    lr = initial_lr * (drop ** np.floor((1 + epoch) / epochs_drop))
    return lr

def cosine_decay(epoch, initial_lr=0.0005, total_epochs=100):
    """
    Cosine decay function for learning rate.
    :param epoch: Current epoch number.
    :param initial_lr: Initial learning rate.
    :param total_epochs: Total number of epochs.
    :return: Adjusted learning rate.
    """
    cosine_decay = 0.5 * (1 + np.cos(np.pi * epoch / total_epochs))
    lr = initial_lr * cosine_decay
    return lr

lr_scheduler = LearningRateScheduler(step_decay)

model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.0005, momentum=0.9),
              loss={'policy': 'categorical_crossentropy', 'value': 'binary_crossentropy'},
              loss_weights={'policy' : 1.0, 'value' : 1.0},
              metrics={'policy': 'categorical_accuracy', 'value': 'mse'})

for i in range (1, epochs + 1):
    print ('epoch ' + str (i))
    golois.getBatch (input_data, policy, value, end, groups, i * N)
    history = model.fit(input_data,
                        {'policy': policy, 'value': value},
                        epochs=1, batch_size=batch
                        ,callbacks=[lr_scheduler]
                        )
    if (i % 5 == 0):
        gc.collect ()
    if (i % epochs == 0):
        golois.getValidation (input_data, policy, value, end)
        val = model.evaluate (input_data,
                              [policy, value], verbose = 0, batch_size=batch)
        print ("val =", val)
        model.save ('test.h5')

epoch 1
79/79 [==============================] - 16s 109ms/step - loss: 6.6425 - policy_loss: 5.8215 - value_loss: 0.8117 - policy_categorical_accuracy: 0.0046 - value_mse: 0.1580
epoch 2
79/79 [==============================] - 8s 102ms/step - loss: 6.3651 - policy_loss: 5.6099 - value_loss: 0.7459 - policy_categorical_accuracy: 0.0129 - value_mse: 0.1385
epoch 3
79/79 [==============================] - 8s 101ms/step - loss: 5.7181 - policy_loss: 4.9866 - value_loss: 0.7222 - policy_categorical_accuracy: 0.0499 - value_mse: 0.1325
epoch 4
79/79 [==============================] - 8s 104ms/step - loss: 5.0849 - policy_loss: 4.3620 - value_loss: 0.7136 - policy_categorical_accuracy: 0.1100 - value_mse: 0.1297
epoch 5
79/79 [==============================] - 8s 104ms/step - loss: 4.9115 - policy_loss: 4.1944 - value_loss: 0.7078 - policy_categorical_accuracy: 0.1242 - value_mse: 0.1261
epoch 6
79/79 [==============================] - 8s 105ms/step - loss: 4.7921 - policy_loss: 4.0794 - va

KeyboardInterrupt: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
results = {'Metrics': ['loss', 'policy_loss', 'value_loss', 'policy_categorical_accuracy', 'value_mse'], 'Prev_val': prev_val, 'val': val}
pd.DataFrame(results)

,Metrics,Prev_val,val
0,loss,3.598189,3.682681
1,policy_loss,2.895999,2.887513
2,value_loss,0.693104,0.690938
3,policy_categorical_accuracy,0.322900,0.327300
4,value_mse,0.120026,0.118950


In [ ]:
prev_val = val.copy()

In [ ]:
model.save('test_{0}.h5'.format(round(val[3],2)))
model.save('/content/drive/MyDrive/Colab Notebooks/Models/David_Benayoun.h5', save_format="h5")


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model = keras.models.load_model('/content/test_0.35.h5')

In [ ]:
model.summary()

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 board (InputLayer)          [(None, 19, 19, 31)]         0         []                            
                                                                                                  
 conv2d_100 (Conv2D)         (None, 19, 19, 32)           1024      ['board[0][0]']               
                                                                                                  
 conv2d_101 (Conv2D)         (None, 19, 19, 32)           9248      ['conv2d_100[0][0]']          
                                                                                                  
 batch_normalization_47 (Ba  (None, 19, 19, 32)           128       ['conv2d_101[0][0]']          
 tchNormalization)                                                                         